In [1]:
%reload_ext autoreload
%autoreload
import sys
sys.path.append("../src")

In [2]:
import os
import pandas as pd
import json
import csv
from tqdm import tqdm

from sapsam import parser, constants
from sapsam.SignavioConventionsChecker import bp_conventions_checker, syntax_checker
from sapsam.SignavioAuthenticator import SignavioAuthenticator

# Read data

Let's first read in the data of a model in the dataset. A subset of the data is used here to prevent unnecessary memory usage.

In [3]:
df = parser.parse_model()
display(df.head())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 732.63it/s]


,revision_id,organization_id,datetime,model_json,description,name,type,namespace
model_id,,,,,,,,
0c4a4701a6524166adaab25fb9e06374,24280a31d8a045aebdbd69b996a848b7,c41660388a04449a995dccf97d894f46,2021-08-14 17:19:56,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,12345678,NaN,http://b3mn.org/stencilset/bpmn2.0#
0c4a6cb2013d488e811de3cca728d5b6,e839d9d1b7244acd8324ff4b05b4cf57,54bfa7a85e5741878102658795743f3a,2021-02-11 12:36:57,"{""resourceId"": ""canvas"", ""properties"": {""orien...",NaN,Lösung zu Übung 1 BKMs Finanzierungsvergleich,NaN,http://signavio.com/stencilsets/dmn-1.0#
0c4a6e19f392466c9f5ea34f0753397d,cb8a1351a515467b822fae0cf0b527df,ea6228bbe6f14163a04b3d115bae768b,2021-04-03 17:01:03,"{""resourceId"": ""canvas"", ""properties"": {""name""...",NaN,homework1,NaN,http://b3mn.org/stencilset/bpmn2.0#
0c4a78ebe86f48aaa7860059c2c117db,a82d4aba3c5d40d39181aab57fe6c368,53efc177fa0a4e98af2fd3bf7be18470,2019-06-06 11:09:06,"{""resourceId"": ""canvas"", ""formats"": {""document...",NaN,Mitarbeiter Onboarding,NaN,http://b3mn.org/stencilset/bpmn2.0#
0c4a80580f0143c691ea5bbbb1f42ba1,188fc2f3e52b429bbda22f12bfb01edc,a54008f9eb7b45129caa83592542548a,2017-07-02 19:11:17,"{""resourceId"": ""canvas"", ""properties"": {""audit...",NaN,Jane Doe:Unterprozess:Auszahlung veranlassen,NaN,http://b3mn.org/stencilset/bpmn2.0#


# Retrieving conventions

To perform the API request for Best Practices (BP) conventions, we need to set up a conf file (see instructions in notebook #3) and obtain the guideline ID of any workspace.

> Note: make sure your are allowed to access this workspace with your credentials by logging into it through the browser first.

To find the guideline ID, one way is to log on to the workspace, perform a manual BP check on a diagram with the mouse, and inspect the payload of the 'mgeditorchecker' HTTP request. The guideline ID can be found in the payload tab.

Here is an example of a guideline ID used in this code:

In [4]:
guideline_id = '4551c2229baa4c79a151b5a0cc1010d2'

The following code extracts the JSON data for the first model in the list, runs the syntax and BP checks through their respective API endpoints, and shows us the result.

In [5]:
auth_data = SignavioAuthenticator.authenticate()                  
model_id = df.index.unique()[0]
model_data = df.loc[model_id]
name = model_data['name']
model_json = model_data['model_json']
syntax_errors = syntax_checker(model_json, auth_data)
violations_count = bp_conventions_checker(name, model_id, guideline_id, model_json, auth_data)
print(f"Syntax errors: {syntax_errors}")
print(f"BP violations count: {violations_count}")

Syntax errors: {"errors": {"sid-C744BE2D-16E8-4E01-9AFD-C3231949A6F6": ["BPMN_MESSAGE_FLOW_NOT_ALLOWED"]}, "warnings": {}}
BP violations count: {"errors": 4, "warnings": 52, "info": 61}
